# TigerGraphX Quick Start: Using TigerGraph for Graph and Vector Database

Follow this guide to quickly get started with TigerGraph for storing both graph data and vectors. This guide assumes that you have already installed TigerGraphX and its dependencies, as outlined in the [Installation Guide](../installation).

To run this Jupyter Notebook, download the original `.ipynb` file from [quick_start_both.ipynb](https://github.com/xuanleilin/tigergraphx/tree/main/docs/getting_started/quick_start_both.ipynb).

---

In this quick start guide, we will work with the following graph:

![Financial Graph](https://raw.githubusercontent.com/tigergraph/ecosys/master/demos/guru_scripts/docker/tutorial/4.x/FinancialGraph.jpg)

## Create a Graph
### Define a Graph Schema
TigerGraph is a schema-based database, which requires defining a schema to structure your graph. This schema specifies the graph name, nodes (vertices), edges (relationships), and their respective attributes.

In this example, we create a graph called "FinancialGraph" with three node types: "Account," "City," and "Phone," and three edge types: "transfer," "hasPhone," and "isLocatedIn."

- **Nodes:**
  - **Account**: Primary key `name`, attributes `name` (string) and `isBlocked` (boolean), vector attribute `emb1` (3).
  - **City**: Primary key `name`, attribute `name` (string).
  - **Phone**: Primary key `number`, attributes `number` (string) and `isBlocked` (boolean), vector attribute `emb1` (3).

- **Edges:**
  - **transfer**: Directed multi-edge between "Account" nodes, identified by `date` (datetime) as the unique identifier for each edge between a pair of source and target nodes. The edge has an attribute `amount` (integer).
  - **hasPhone**: Undirected edge between "Account" and "Phone" nodes.
  - **isLocatedIn**: Directed edge between "Account" and "City" nodes.

This schema defines the structure of the "FinancialGraph" with nodes and edges and their respective attributes.

In [1]:
graph_schema = {
    "graph_name": "FinancialGraph",
    "nodes": {
        "Account": {
            "primary_key": "name",
            "attributes": {
                "name": "STRING",
                "isBlocked": "BOOL",
            },
            "vector_attributes": {"emb1": 3},
        },
        "City": {
            "primary_key": "name",
            "attributes": {
                "name": "STRING",
            },
        },
        "Phone": {
            "primary_key": "number",
            "attributes": {
                "number": "STRING",
                "isBlocked": "BOOL",
            },
            "vector_attributes": {"emb1": 3},
        },
    },
    "edges": {
        "transfer": {
            "is_directed_edge": True,
            "from_node_type": "Account",
            "to_node_type": "Account",
            "discriminator": "date",
            "attributes": {
                "date": "DATETIME",
                "amount": "INT",
            },
        },
        "hasPhone": {
            "is_directed_edge": False,
            "from_node_type": "Account",
            "to_node_type": "Phone",
        },
        "isLocatedIn": {
            "is_directed_edge": True,
            "from_node_type": "Account",
            "to_node_type": "City",
        },
    },
}

### Define the TigerGraph Connection Configuration
In addition to defining the schema, a connection configuration is necessary to establish communication with the TigerGraph server.

In [2]:
connection = {
    "host": "http://127.0.0.1",
    "username": "tigergraph",
    "password": "tigergraph",
}

### Create a Graph
Running the following command will create a graph using the user-defined schema if it does not already exist. If the graph exists, the command will return the existing graph. To overwrite the existing graph, set the drop_existing_graph parameter to True. Note that creating the graph may take several seconds.

In [3]:
from tigergraphx import Graph
G = Graph(graph_schema, connection)

2025-01-06 21:53:00,459 - tigergraphx.core.graph.base_graph - INFO - Creating schema for graph FinancialGraph...
2025-01-06 21:54:03,285 - tigergraphx.core.graph.base_graph - INFO - Schema created successfully.


## Add Add Nodes and Edges
*Note*: This example demonstrates how to easily add nodes and edges using the API. However, adding nodes and edges individually may not be efficient for large-scale operations. For better performance when loading data into TigerGraph, it is recommended to use a loading job. Nonetheless, these examples are ideal for quickly getting started.
### Add Nodes

In [4]:
nodes_for_adding = [
    ("Scott", {"isBlocked": False}),
    ("Jenny", {"isBlocked": False}),
    ("Steven", {"isBlocked": True}),
    ("Paul", {"isBlocked": False}),
    ("Ed", {"isBlocked": False}),
]
print(G.add_nodes_from(nodes_for_adding, node_type="Account"))
nodes_for_adding = [
    ("718-245-5888", {"isBlocked": False}),
    ("650-658-9867", {"isBlocked": True}),
    ("352-871-8978", {"isBlocked": False}),
]
print(G.add_nodes_from(nodes_for_adding, node_type="Phone"))
nodes_for_adding = ["New York", "Gainesville", "San Francisco"]
print(G.add_nodes_from(nodes_for_adding, node_type="City"))

5
3
3


### Add Edges

In [5]:
ebunch_to_add = [
    ("Scott", "718-245-5888"),
    ("Jenny", "718-245-5888"),
    ("Jenny", "650-658-9867"),
    ("Paul", "650-658-9867"),
    ("Ed", "352-871-8978"),
]
print(G.add_edges_from(ebunch_to_add, "Account", "hasPhone", "Phone"))
ebunch_to_add = [
    ("Scott", "New York"),
    ("Jenny", "San Francisco"),
    ("Steven", "San Francisco"),
    ("Paul", "Gainesville"),
    ("Ed", "Gainesville"),
]
print(G.add_edges_from(ebunch_to_add, "Account", "isLocatedIn", "City"))
ebunch_to_add = [
    ("Scott", "Ed", {"date": "2024-01-04", "amount": 20000}),
    ("Scott", "Ed", {"date": "2024-02-01", "amount": 800}),
    ("Scott", "Ed", {"date": "2024-02-14", "amount": 500}),
    ("Jenny", "Scott", {"date": "2024-04-04", "amount": 1000}),
    ("Paul", "Jenny", {"date": "2024-02-01", "amount": 653}),
    ("Steven", "Jenny", {"date": "2024-05-01", "amount": 8560}),
    ("Ed", "Paul", {"date": "2024-01-04", "amount": 1500}),
    ("Paul", "Steven", {"date": "2023-05-09", "amount": 20000}),
]
print(G.add_edges_from(ebunch_to_add, "Account", "transfer", "Account"))

5
5
8


### Inserting Embeddings into Nodes
To insert embeddings into the nodes, you can use the following data format:

In [6]:
data = [
    {
        "name": "Scott",
        "emb1": [-0.017733968794345856, -0.01019224338233471, -0.016571875661611557],
    },
    {
        "name": "Jenny",
        "emb1": [-0.019265105947852135, 0.0004929182468913496, 0.006711316294968128],
    },
    {
        "name": "Steven",
        "emb1": [-0.01505514420568943, -0.016819344833493233, -0.0221870020031929],
    },
    {
        "name": "Paul",
        "emb1": [0.0011193430982530117, -0.001038988004438579, -0.017158523201942444],
    },
    {
        "name": "Ed",
        "emb1": [-0.003692442551255226, 0.010494389571249485, -0.004631792660802603],
    },
]
print(G.upsert(data, "Account"))

0


Note: A result of 0 simply indicates that no new nodes were added, as the nodes were already inserted into TigerGraph. This does not mean the update failed.

### Display the Number of Nodes
Next, let's verify that the data has been inserted into the graph by using the following command. As expected, the number of nodes is 5.

In [7]:
print(G.number_of_nodes())

11


## Perform Vector Search
To find the top 2 most similar accounts to "Scott" based on the embedding, we use the following code. As expected, "Scott" will appear in the list with a distance of 0.

In [8]:
data = [-0.017733968794345856, -0.01019224338233471, -0.016571875661611557]
print(G.search(data, vector_attribute_name="emb1", node_type="Account", limit=2))

[{'id': 'Paul', 'distance': 0.393388, 'name': 'Paul', 'isBlocked': False}, {'id': 'Steven', 'distance': 0.0325563, 'name': 'Steven', 'isBlocked': True}]


## Clear and Drop a Graph

### Clear the Graph
To clear the data in the graph without dropping it, use the following code:

In [9]:
print(G.clear())

True


Afterwards, you can confirm that there are no nodes in the graph by checking:

In [10]:
print(G.number_of_nodes())

0


### Drop the Graph
To clear the data and completely remove the graph—including schema, loading jobs, and queries—use the following code:

In [11]:
G.drop_graph()

---

## What’s Next?

Now that you've learned how to use TigerGraph for storing both graph data and vectors, you can dive into more advanced features of TigerGraphX:
- [GraphRAG Overview](../../graphrag/graphrag_overview): Learn about integrating graphs with LLMs.
- [API Reference](../../reference/features_overview): Dive deeper into TigerGraphX APIs.

---

Start unlocking the power of graphs with **TigerGraphX** today!